In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import re, string, unicodedata
import os
import inflect
import numpy as np
import pandas as pd
import collections
import networkx as nx
import time
import pickle

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk import download
from nltk.corpus import stopwords


from sklearn.metrics.pairwise import cosine_similarity


%pip install stanza
import stanza


     |████████████████████████████████| 342 kB 5.4 MB/s 


In [3]:
download('stopwords')
download('punkt')
stanza.download('es')
nlp = stanza.Pipeline('es')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2021-09-08 00:02:51 INFO: Downloading default packages for language: es (Spanish)...


2021-09-08 00:05:04 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-09-08 00:05:04 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-08 00:05:04 INFO: Use device: cpu
2021-09-08 00:05:04 INFO: Loading: tokenize
2021-09-08 00:05:04 INFO: Loading: mwt
2021-09-08 00:05:04 INFO: Loading: pos
2021-09-08 00:05:04 INFO: Loading: lemma
2021-09-08 00:05:05 INFO: Loading: depparse
2021-09-08 00:05:05 INFO: Loading: ner
2021-09-08 00:05:06 INFO: Done loading processors!


PREPROCESADO DEL TWEET

In [4]:
def eliminar_textos_basura(texto):
    # eliminar emojis: elimina todo lo que este en <>. 
    texto_procesado = re.sub('<.*?>', ' ', texto)
    # convertir a minusculas
    texto_procesado = texto_procesado.lower()
    # remover @usuario
    texto_procesado = re.sub('@[^\s]+',' ',texto_procesado)
    # remover RT
    texto_procesado = re.sub('rt   ',' ',texto_procesado)
    texto_procesado = re.sub('rt',' ',texto_procesado)
    texto_procesado = re.sub('rt ',' ',texto_procesado)
    # remover numeros
    texto_procesado = re.sub("\d+", "", texto_procesado)
    # texto_procesado = re.sub(r'http\S+', ' ', texto_procesado)
    texto_procesado = re.sub("(\w+:\/\/\S+)", " ", texto_procesado)
    # reemplazar todos los caracteres que no saen alfanuméricos con espacios
    texto_procesado = re.sub(r'[^a-zA-Z0-9ÑñÁáÉéÍíÓóÚú\s]', ' ', texto_procesado)
    
    return texto_procesado

In [5]:
def eliminar_stop_words(texto):
    # obtener lista de stopwords
    stop_words = set(stopwords.words('spanish')) 
    stop_words.remove('no')
    #separar el texto por palabras
    palabras = word_tokenize(texto) 
    # dejar solo palabras que no sean stopwords
    texto_sin_sw = [w for w in palabras if not w in stop_words] 
    #  convertir en una sola cadena la lista de palabras
    texto_sin_sw = ' '.join(texto_sin_sw)

    return texto_sin_sw 


In [6]:
def remove_non_ascii(word):
    """Remueve caracteres no ASCII"""
    new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    return new_word


In [7]:
def establecer_lema(text):
  doc = nlp(text)
  new = ''
  for sent in doc.sentences:
    for word in sent.words:
      new = new + word.lemma
      new = new + ' '

  return new

In [8]:
def ProcesarTweet(Tweet):
  Procesado = eliminar_textos_basura(Tweet)
  Procesado = eliminar_stop_words(Procesado)
  Procesado = remove_non_ascii(Procesado)
  Procesado = establecer_lema(Procesado)
  return Procesado

FUNCIONES PARA DETERMINAR LA NUEVA ETIQUETA

In [9]:
tipo = [] #Variable que replicara los valores de la columna tipo del datset usado para entrenar los modelos
for i in range(1290):
  tipo.append(1)
for j in range(1380):
  tipo.append(0)

In [10]:
def Knn(Matriz, k):
  labels = tipo
  pred_label = []
  etiquetar = []
  indice = 0
  for x in labels:
      if(indice != len(labels)-1):
        if(Matriz[indice]<=0.25): #Filtro que asigan directament 0 cuando el valor de comparacion sea cero o menor a cero
          etiquetar.append((Matriz[indice], 0))
        else:
          etiquetar.append((Matriz[indice], x))
      indice += 1
  etiquetar.sort(reverse=True)
  neighbors = etiquetar[:k]
  votes = []
  for neighbor in neighbors:
            votes.append(neighbor[1])
  counter = collections.Counter(votes) #Determina el que mas se repite
  pred_label.append(counter.most_common()[0][0])
  return pred_label

In [11]:
def EtiquetaEnTexto(valor):
  if valor[0]==1:
    return 'emergencia'
  else:
    return 'no emergencia'

**MODELOS**

MODELO COSENO

In [12]:
#CARGAR MODELO COSENO

# Cargamos el vectorizer
vectorizer = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/N_VectorizerCOSENO.pickle','rb'))
# Cargamos la data vectorizada
DataVectCOSENO = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/N_DataVectoCOSENO.pickle','rb'))


In [13]:
def metodo_coseno(tweet):
  Tweet = ProcesarTweet(tweet) #Preprocesamiento del tweet
  if(len(Tweet) == 0): #En el caso que al preprocesar no queden palabras, se le asigna una palabra
    Tweet = 'vacio'
  TweetVect = vectorizer.transform([Tweet]) #Se convierte el tweet en el formato de los tweets del modelo
  matriz = []
  for x in DataVectCOSENO:
    matriz.append(cosine_similarity(x,TweetVect)[0][0]) #Se compara el nuevo tweet con cada tweet de la dataset
  return matriz

In [14]:
#Función para usar el modelo
def EtiquetarModelCOSENO(tweet):
  Similaridad= metodo_coseno(tweet)
  k_1 = Knn(Similaridad, 1)
  k_3 = Knn(Similaridad, 3)
  k_5 = Knn(Similaridad, 5)
  k_7 = Knn(Similaridad, 7)
  k_9 = Knn(Similaridad, 9)
  k_11 = Knn(Similaridad, 11)
  return k_1, k_3, k_5, k_7, k_9, k_11 

MODELO EMPLEANDO TFIDF

In [15]:
#Cargar Modelo
import pickle

modelTFIDF = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/N_modelTFIDF.model','rb'))
DiccioTFIDF = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/N_DiccionarioTFIDF.pickle','rb'))
MatrizSimTFIDF = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/N_MatrizSimilaridadTFIDF.pickle','rb'))

In [16]:
def metodo_TFIDF(tweet):
  Tweet = word_tokenize(ProcesarTweet(tweet)) #Se Preprocesa el tweet y se tokeniza el texto
  
  if len(Tweet) == 0:  #Se valida que el tweet preprocesado tenga palabras
    Tweet = ['vacio']  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia
  
  kw_vector = DiccioTFIDF.doc2bow(Tweet)        #Se estructura el tweet en función al diccionario del modelo
  sim = MatrizSimTFIDF[modelTFIDF[kw_vector]]   #Se genera un nuevo vector que contiene el nivel de similitud del nuevo tweet con cada tweet del dataset

  Valor_Similitud = []
  for i in range(len(sim)):
    a = ('%.2f' % sim[i]) #Se redondea cada valor del vector de similitud creado para establecer un mismo formato a todos los valores
    Valor_Similitud.append(float(a))

  return Valor_Similitud

In [17]:
def EtiquetarModelTFIDF(tweet):
  Similaridad = metodo_TFIDF(tweet)
  k_1 = Knn(Similaridad, 1)
  k_3 = Knn(Similaridad, 3)
  k_5 = Knn(Similaridad, 5)
  k_7 = Knn(Similaridad, 7)
  k_9 = Knn(Similaridad, 9)
  k_11 = Knn(Similaridad, 11)
  return k_1, k_3, k_5, k_7, k_9, k_11 

Modelo LSI

In [18]:
#Cargar Modelo
import pickle

modelLSI = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/ModelLSI0.model','rb'))
DiccioLSI = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/DiccionarioLSI0.pickle','rb'))
MatrizSimLSI = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/MatrizSimilaridadLSI0.pickle','rb'))

In [19]:
def modeloLSI(tweet):
  Tweet = word_tokenize(ProcesarTweet(tweet)) #Se Preprocesa el tweet y se tokeniza el texto
  
  if len(Tweet) == 0: #Se valida que el tweet preprocesado tenga palabras
    Tweet = ['vacio']  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia
  
  tweet_Dic = DiccioLSI.doc2bow(Tweet) #Se estructura el tweet en función al diccionario del modelo
  sim = MatrizSimLSI[modelLSI[tweet_Dic]] #Se genera un nuevo vector que contiene el nivel de similitud del nuevo tweet con cada tweet del dataset
  
  Valor_Similitud = [] 
  for i in range(len(sim)):
    a = ('%.2f' % sim[i]) #Se redondea cada valor del vector de similitud creado para establecer un mismo formato a todos los valores
    Valor_Similitud.append(float(a))
  
  return Valor_Similitud

In [20]:
def EtiquetarModelLSI(tweet):
  Similaridad= modeloLSI(tweet)
  k_1 = Knn(Similaridad, 1)
  k_3 = Knn(Similaridad, 3)
  k_5 = Knn(Similaridad, 5)
  k_7 = Knn(Similaridad, 7)
  k_9 = Knn(Similaridad, 9)
  k_11 = Knn(Similaridad, 11)
  return k_1, k_3, k_5, k_7, k_9, k_11 

PRUEBA INDIVIDUAL PARA LOS MODELOS

In [21]:
Tweet = 'Hubo un choque de vehiculos, ayuda hay heridos y muertos' #COLOCAR TWEET
Etiqueta = EtiquetarModelLSI(Tweet) #INDIQUE EL MODELO
print('RESULTADOS DEL MODELO EN BASE AL TWEET: ',Tweet )
print('Etiqueta K=1 : ', EtiquetaEnTexto(Etiqueta[0]) )
print('Etiqueta K=3 : ', EtiquetaEnTexto(Etiqueta[1]))
print('Etiqueta K=5 : ', EtiquetaEnTexto(Etiqueta[2]))
print('Etiqueta K=7 : ', EtiquetaEnTexto(Etiqueta[3]))

RESULTADOS DEL MODELO EN BASE AL TWEET:  Hubo un choque de vehiculos, ayuda hay heridos y muertos
Etiqueta K=1 :  no emergencia
Etiqueta K=3 :  no emergencia
Etiqueta K=5 :  emergencia
Etiqueta K=7 :  emergencia


VALIDACIÓN

In [22]:
import pandas as pd
import numpy as np
import collections
from sklearn import datasets, metrics

In [23]:
def GenerarResultadosModelo(DataN, modelo):
  Data = DataN
  Reemplazo = Data['tipo'].replace(['emergencia'], 1).replace(['no emergencia'], 0)
  Data['tipo'] = Reemplazo
  text = list(Data['New_Tweet'])
  
  prediccion1 = []
  prediccion2 = []
  prediccion3 = []
  prediccion4 = []
  prediccion5 = []
  prediccion6 = []

  for a in text:
    metodo= modelo(a)
    prediccion1.append(metodo[0])
    prediccion2.append(metodo[1])
    prediccion3.append(metodo[2])
    prediccion4.append(metodo[3])
    prediccion5.append(metodo[4])
    prediccion6.append(metodo[5])
  test_final = list(Data['tipo'])

  return  test_final, prediccion1, prediccion2, prediccion3, prediccion4, prediccion5, prediccion6 

In [24]:
def EmplearMetricasDeValidacion(DataPrueba, Modelo, columnas):
  metric1 = []
  metric2 = []
  metric3 = []
  metric4 = []
  metric5 = []

  re = GenerarResultadosModelo(DataPrueba, Modelo)
  #k1
  metric1.append(metrics.accuracy_score(re[0], re[1]))
  metric2.append(metrics.precision_score(re[0], re[1]))
  metric3.append(metrics.recall_score(re[0], re[1]))
  #metric4.append(metrics.f1_score(re[0], re[1]))
  metric5.append(metrics.log_loss(re[0], re[1]))
  #k3
  metric1.append(metrics.accuracy_score(re[0], re[2]))
  metric2.append(metrics.precision_score(re[0], re[2]))
  metric3.append(metrics.recall_score(re[0], re[2]))
  #metric4.append(metrics.f1_score(re[0], re[2]))
  metric5.append(metrics.log_loss(re[0], re[2]))
  #k5
  metric1.append(metrics.accuracy_score(re[0], re[3]))
  metric2.append(metrics.precision_score(re[0], re[3]))
  metric3.append(metrics.recall_score(re[0], re[3]))
  #metric4.append(metrics.f1_score(re[0], re[3]))
  metric5.append(metrics.log_loss(re[0], re[3]))
  #k7
  metric1.append(metrics.accuracy_score(re[0], re[4]))
  metric2.append(metrics.precision_score(re[0], re[4]))
  metric3.append(metrics.recall_score(re[0], re[4]))
  #metric4.append(metrics.f1_score(re[0], re[4]))
  metric5.append(metrics.log_loss(re[0], re[4]))

  #k9
  metric1.append(metrics.accuracy_score(re[0], re[5]))
  metric2.append(metrics.precision_score(re[0], re[5]))
  metric3.append(metrics.recall_score(re[0], re[5]))
  #metric4.append(metrics.f1_score(re[0], re[5]))
  metric5.append(metrics.log_loss(re[0], re[5]))

  #k11
  metric1.append(metrics.accuracy_score(re[0], re[6]))
  metric2.append(metrics.precision_score(re[0], re[6]))
  metric3.append(metrics.recall_score(re[0], re[6]))
  #metric4.append(metrics.f1_score(re[0], re[6]))
  metric5.append(metrics.log_loss(re[0], re[6]))
  
  resultado=[metric1,metric2,metric3,metric5]

  Final = pd.DataFrame(resultado,columns=columnas)
  Final.insert(0,"METRICAS VALIDACION",['ACURRACY_SCORE','PRECISIÓN_SCORE','RECALL_SCORE','LOG_LOSS'],True)
  
  return Final

In [25]:
def GenerarReporteModelos(Data):
   print('COSENO en Proceso')
   Coseno = EmplearMetricasDeValidacion(Data, EtiquetarModelCOSENO, ['coseno_1','coseno_3','coseno_5','coseno_7','coseno_9','coseno_11'])
   print('TFIDF en Proceso')
   TFIDF  = EmplearMetricasDeValidacion(Data, EtiquetarModelTFIDF, ['Tfidf_1','Tfidf_3','Tfidf_5','Tfidf_7','Tfidf_9','Tfidf_11'])
   print('LSI en Proceso')
   LSI    = EmplearMetricasDeValidacion(Data, EtiquetarModelLSI, ['lsi_1','lsi_3','lsi_5','lsi_7','lsi_9','lsi_11'])
   
   a = pd.merge(Coseno, TFIDF,  on='METRICAS VALIDACION')
   b = pd.merge(a, LSI,  on='METRICAS VALIDACION')

   return b

In [26]:
def GenerarInformeResultadosModelo(Data,modelo):
  text = list(Data['New_Tweet'])
  re = GenerarResultadosModelo(Data, modelo)
  resultado=[text,re[0],re[1],re[2],re[3],re[4],re[5],re[6]]
  Final = pd.DataFrame(resultado)
  return Final

GENERAR REPORTES

In [27]:
DataNivelada = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Tweets_Etiquetados/New_DataPrueba_Total.xlsx')

In [28]:
#REPORTE DETALLADO DE TODOS LOS MODELOS
ReporteMODELOS=GenerarReporteModelos(DataNivelada)
ReporteMODELOS

COSENO en Proceso


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


TFIDF en Proceso
LSI en Proceso


,METRICAS VALIDACION,coseno_1,coseno_3,coseno_5,coseno_7,coseno_9,coseno_11,Tfidf_1,Tfidf_3,Tfidf_5,Tfidf_7,Tfidf_9,Tfidf_11,lsi_1,lsi_3,lsi_5,lsi_7,lsi_9,lsi_11
0,ACURRACY_SCORE,0.796250,0.808750,0.777500,0.746250,0.731250,0.720000,0.721250,0.642500,0.603750,0.576250,0.563750,0.550000,0.816250,0.867500,0.886250,0.892500,0.876250,0.855000
1,PRECISIÓN_SCORE,0.809659,0.867742,0.858657,0.853175,0.840164,0.855204,0.780142,0.862595,0.872093,0.838710,0.820000,0.769231,0.788177,0.851662,0.875648,0.893333,0.885246,0.886297
2,RECALL_SCORE,0.748031,0.706037,0.637795,0.564304,0.538058,0.496063,0.577428,0.296588,0.196850,0.136483,0.107612,0.078740,0.839895,0.874016,0.887139,0.879265,0.850394,0.797900
3,LOG_LOSS,7.037343,6.605582,7.684918,8.764251,9.282335,9.670889,9.627746,12.347631,13.686001,14.635816,15.067550,15.542458,6.346586,4.576446,3.928834,3.712958,4.274216,5.008162


In [29]:
#ALMACENAR REPORTE

ReporteMODELOS.to_excel('Reporte_Modelos_newData2.xlsx', index=False)
!cp Reporte_Modelos_newData2.xlsx '/content/gdrive/My Drive/Colab Notebooks/Reporte_tweets_Ecuador/'


In [ ]:
#REPORTE DETALLADO DEL MEJOR MODELO : LSI
ReporteDelModelo = GenerarInformeResultadosModelo(DataNivelada, EtiquetarModelLSI)
ReporteDelModelo.T

,0,1,2,3,4,5,6,7
0,"RT @DiarioExtraEc: Freddy Carrión, defensor de...",0,[0],[0],[0],[0],[0],[0]
1,RT @CHEL01977: @LassoGuillermo No es aceptable...,0,[0],[0],[0],[0],[0],[0]
2,RT @edicionmedEC: 📌 Defensoría del Pueblo plan...,0,[0],[0],[0],[0],[0],[0]
3,RT @elcomerciocom: Las vacunas contra el covid...,0,[1],[1],[0],[0],[0],[0]
4,RT @ecuavisa: Fue clausurado en el sur de Quit...,0,[1],[1],[1],[1],[1],[1]
...,...,...,...,...,...,...,...,...
69,URGENTE: No llegan los bomberos al incendio,1,[1],[1],[1],[1],[1],[1]
70,Ayuda !! una persona esta en peligro de caer d...,1,[0],[1],[0],[0],[0],[0]
71,Se escuharon 3 disparos en el centro comercial...,1,[0],[0],[1],[1],[1],[1]
72,Persona herida despues de robo a mano armada e...,1,[1],[1],[1],[1],[1],[1]


In [ ]:
#ALMACENAR REPORTE
'''
ReporteDelModelo.T.to_excel('VerificacionModelo_LSI_new.xlsx', index=False)
!cp VerificacionModelo_LSI_new.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/'
'''

"\nReporteDelModelo.T.to_excel('VerificacionModelo_LSI_new.xlsx', index=False)\n!cp VerificacionModelo_LSI_new.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/'\n"